<a href="https://colab.research.google.com/github/HegdeSiddesh/cs6910_Assignment3/blob/main/ASG3_Q2_sweep_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
!pip install --upgrade wandb
import wandb
from wandb.keras import WandbCallback

     |████████████████████████████████| 1.8 MB 7.9 MB/s 
     |████████████████████████████████| 181 kB 93.0 MB/s 
     |████████████████████████████████| 144 kB 99.0 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=70c19dcae8455f72b7ef877596fc776741ad40c19ba84369c464d500d63852b8
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [ ]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
#Download and unzip the Dakshina dataset 
!wget https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar
!tar -xf '/content/dakshina_dataset_v1.0.tar'

--2022-04-30 06:52:40--  https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.99.128, 142.250.107.128, 74.125.199.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.99.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2008340480 (1.9G) [application/x-tar]
Saving to: ‘dakshina_dataset_v1.0.tar’

dakshina_dataset_v1 100%[===================>]   1.87G   151MB/s    in 13s     

2022-04-30 06:52:53 (148 MB/s) - ‘dakshina_dataset_v1.0.tar’ saved [2008340480/2008340480]



In [ ]:
wandb.init(project="CS6910-Assignment_3-sweep-Tamil", entity="hithesh-sidhesh", name="Question_2")

wandb: Currently logged in as: hithesh-sidhesh (use `wandb login --relogin` to force relogin)


In [ ]:
#Read the contents from the csv file. Here we have chosen english to tamil transliteration.

def load_data(path):
    with open(path) as fil:
        data = pd.read_csv(fil,sep='\t',header=None,names=["Tamil","English",""],skip_blank_lines=True,index_col=None)
    data = data[data['Tamil'].notna()] # using notna() ensures 'nan' are deleted from the data. If not it will throw an error while training.
    data = data[data['English'].notna()]
    data = data[['Tamil','English']]
    return data

train = load_data("/content/dakshina_dataset_v1.0/ta/lexicons/ta.translit.sampled.train.tsv")
val = load_data("/content/dakshina_dataset_v1.0/ta/lexicons/ta.translit.sampled.dev.tsv")
test = load_data("/content/dakshina_dataset_v1.0/ta/lexicons/ta.translit.sampled.test.tsv")


In [ ]:
x = train['English'].values #Get the english words alone
y = train['Tamil'].values #Get the tamil words alone
# As y = f(x) given , we assign english words to x and tamil words to y. We are training the seq2seq model to transliterate from english to tamil.

# We use "tab" as the "start sequence" character for the targets, and "\n" as "end sequence" character.
y = "\t"+y+"\n"

In [ ]:
input_characters = set()
target_characters = set()

for i in range(len(x)):
  input_characters.update(list(str(x[i])))
  target_characters.update(list(str(y[i])))

#We get the unique characters(alphabets) present in the source and target language.

  # input_characters=list(set(input_characters+list(str(x[i]))))
  # target_characters=list(set(target_characters+list(str(y[i]))))

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
#Sorting the alphabets present in the source and target language.

# add the space character to both
input_characters.append(" ")
target_characters.append(" ")

#Getting the total number of alphabets present in the source and target language 
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

input_length=[]
target_length=[]
for i in range(len(x)):
  input_length.append(len(str(x[i])))
  target_length.append(len(str(y[i])))

#Getting the maximum length of the word present in the source and target language
max_encoder_seq_length = max(input_length)
max_decoder_seq_length = max(target_length)

print("Number of samples:", len(x))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)


#Making a dictionary for the alphabets(unique characters) present in the source and target language
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])
print(input_token_index)
print(target_token_index)

#Defining tensor for input and output data.
encoder_input_data = np.zeros((len(x), max_encoder_seq_length, num_encoder_tokens), dtype="float32")
decoder_input_data = np.zeros((len(x), max_decoder_seq_length, num_decoder_tokens), dtype="float32")
decoder_target_data = np.zeros((len(x), max_decoder_seq_length, num_decoder_tokens), dtype="float32")

#Getting one-hot representrations for each word.
for i, (input_text, target_text) in enumerate(zip(x, y)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and hence will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

Number of samples: 68215
Number of unique input tokens: 27
Number of unique output tokens: 49
Max sequence length for inputs: 30
Max sequence length for outputs: 28
{'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8, 'j': 9, 'k': 10, 'l': 11, 'm': 12, 'n': 13, 'o': 14, 'p': 15, 'q': 16, 'r': 17, 's': 18, 't': 19, 'u': 20, 'v': 21, 'w': 22, 'x': 23, 'y': 24, 'z': 25, ' ': 26}
{'\t': 0, '\n': 1, 'ஃ': 2, 'அ': 3, 'ஆ': 4, 'இ': 5, 'ஈ': 6, 'உ': 7, 'ஊ': 8, 'எ': 9, 'ஏ': 10, 'ஐ': 11, 'ஒ': 12, 'ஓ': 13, 'க': 14, 'ங': 15, 'ச': 16, 'ஜ': 17, 'ஞ': 18, 'ட': 19, 'ண': 20, 'த': 21, 'ந': 22, 'ன': 23, 'ப': 24, 'ம': 25, 'ய': 26, 'ர': 27, 'ற': 28, 'ல': 29, 'ள': 30, 'ழ': 31, 'வ': 32, 'ஷ': 33, 'ஸ': 34, 'ஹ': 35, 'ா': 36, 'ி': 37, 'ீ': 38, 'ு': 39, 'ூ': 40, 'ெ': 41, 'ே': 42, 'ை': 43, 'ொ': 44, 'ோ': 45, 'ௌ': 46, '்': 47, ' ': 48}


In [ ]:
#Embedding validation data
x_val = val['English'].values
y_val = val['Tamil'].values
# We use "tab" as the "start sequence" character for the targets, and "\n" as "end sequence" character.
y_val = "\t"+y_val+"\n"

In [ ]:
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [ ]:
val_input_characters = set()
val_target_characters = set()

for i in range(len(x_val)):
  val_input_characters.update(list(str(x_val[i])))
  val_target_characters.update(list(str(y_val[i])))

  # input_characters=list(set(input_characters+list(str(x[i]))))
  # target_characters=list(set(target_characters+list(str(y[i]))))

val_input_characters = sorted(list(val_input_characters))
val_target_characters = sorted(list(val_target_characters))

# add the space character to both
val_input_characters.append(" ")
val_target_characters.append(" ")

val_num_encoder_tokens = len(val_input_characters)
val_num_decoder_tokens = len(val_target_characters)

val_input_length=[]
val_target_length=[]
for i in range(len(x_val)):
  val_input_length.append(len(str(x_val[i])))
  val_target_length.append(len(str(y_val[i])))

val_max_encoder_seq_length = max(val_input_length)
val_max_decoder_seq_length = max(val_target_length)

print("Number of samples in validation set:", len(x_val))
print("Number of unique input tokens:", val_num_encoder_tokens)
print("Number of unique output tokens:", val_num_decoder_tokens)
print("Max sequence length for validation inputs:", val_max_encoder_seq_length)
print("Max sequence length for valiation outputs:", val_max_decoder_seq_length)

val_input_token_index = dict([(char, i) for i, char in enumerate(val_input_characters)])
val_target_token_index = dict([(char, i) for i, char in enumerate(val_target_characters)])
print(val_input_token_index)
print(val_target_token_index)

val_encoder_input_data = np.zeros((len(x_val), val_max_encoder_seq_length,val_num_encoder_tokens), dtype="float32")
val_decoder_input_data = np.zeros((len(x_val), val_max_decoder_seq_length,val_num_decoder_tokens), dtype="float32")
val_decoder_target_data = np.zeros((len(x), max_decoder_seq_length, val_num_decoder_tokens), dtype="float32")

for i, (input_text, target_text) in enumerate(zip(x_val, y_val)):
    for t, char in enumerate(input_text):
        val_encoder_input_data[i, t ,input_token_index[char]] = 1.0
    val_encoder_input_data[i, t + 1 :,input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        val_decoder_input_data[i, t ,target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            val_decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    val_decoder_input_data[i, t + 1 :,target_token_index[" "]] = 1.0
    val_decoder_target_data[i, t:, target_token_index[" "]] = 1.0

Number of samples in validation set: 6827
Number of unique input tokens: 27
Number of unique output tokens: 49
Max sequence length for validation inputs: 23
Max sequence length for valiation outputs: 22
{'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8, 'j': 9, 'k': 10, 'l': 11, 'm': 12, 'n': 13, 'o': 14, 'p': 15, 'q': 16, 'r': 17, 's': 18, 't': 19, 'u': 20, 'v': 21, 'w': 22, 'x': 23, 'y': 24, 'z': 25, ' ': 26}
{'\t': 0, '\n': 1, 'ஃ': 2, 'அ': 3, 'ஆ': 4, 'இ': 5, 'ஈ': 6, 'உ': 7, 'ஊ': 8, 'எ': 9, 'ஏ': 10, 'ஐ': 11, 'ஒ': 12, 'ஓ': 13, 'க': 14, 'ங': 15, 'ச': 16, 'ஜ': 17, 'ஞ': 18, 'ட': 19, 'ண': 20, 'த': 21, 'ந': 22, 'ன': 23, 'ப': 24, 'ம': 25, 'ய': 26, 'ர': 27, 'ற': 28, 'ல': 29, 'ள': 30, 'ழ': 31, 'வ': 32, 'ஷ': 33, 'ஸ': 34, 'ஹ': 35, 'ா': 36, 'ி': 37, 'ீ': 38, 'ு': 39, 'ூ': 40, 'ெ': 41, 'ே': 42, 'ை': 43, 'ொ': 44, 'ோ': 45, 'ௌ': 46, '்': 47, ' ': 48}


In [ ]:
# Using label encoding for the encoder inputs (and then find an embedding using the Embedding layer)
encoder_input_data = np.argmax(encoder_input_data, axis=2)
val_encoder_input_data = np.argmax(val_encoder_input_data, axis=2)
#test_encoder_input_data = np.argmax(test_encoder_input_data, axis=2)
print(encoder_input_data.shape)

(68215, 30)


In [ ]:
decoder_input_data = np.argmax(decoder_input_data, axis=2)
val_decoder_input_data = np.argmax(val_decoder_input_data, axis=2)
#test_decoder_input_data = np.argmax(test_decoder_input_array, axis=2)

In [ ]:
#Build the model
def create_model(num_encoder_tokens,embedding_size,cell_type,latent_dimension,dropout,number_of_encoder_layers,num_decoder_tokens,number_of_decoder_layers):

  # Define an input sequence and process it.
  #Add one or more layers at the encoder and decoder side.
  encoder_inputs = keras.Input(shape=(None,), name='encoder_input')
  encoder = None
  encoder_outputs = None
  state_h = None
  state_c = None
  en_embed = tf.keras.layers.Embedding(input_dim=num_encoder_tokens, output_dim=embedding_size,
                                            name='encoder_embedding')(encoder_inputs)
  if cell_type == 'rnn':
      encoder = keras.layers.SimpleRNN(latent_dimension, return_state=True, return_sequences=True,
                                       name='encoder_hidden_1', dropout=dropout)
      encoder_outputs, state_h = encoder(en_embed)
  elif cell_type == 'gru':
      encoder = keras.layers.GRU(latent_dimension, return_state=True, return_sequences=True,
                                  name='encoder_hidden_1', dropout=dropout)
      encoder_outputs, state_h = encoder(en_embed)
  else:
      encoder = keras.layers.LSTM(latent_dimension, return_state=True, return_sequences=True,
                                  name='encoder_hidden_1', dropout=dropout)
      encoder_outputs, state_h, state_c = encoder(en_embed)

  # First encoder layer


  e_layer = number_of_encoder_layers
  for i in range(2, e_layer + 1):
    # Give the output sequences as input to the next layer 
    
    # Last state is set as initial state of next layer
    layer_name = ('encoder_hidden_%d') % i
    if cell_type == 'rnn':
      encoder = keras.layers.SimpleRNN(latent_dimension, return_state=True, return_sequences=True,
                                          name=layer_name, dropout=dropout)
      encoder_outputs, state_h = encoder(encoder_outputs, initial_state=[state_h])
    elif cell_type == 'gru':
        encoder = keras.layers.GRU(latent_dimension, return_state=True, return_sequences=True,
                                    name=layer_name, dropout=dropout)
        encoder_outputs, state_h = encoder(encoder_outputs, initial_state=[state_h])
    else:
        encoder = keras.layers.LSTM(latent_dimension, return_state=True, return_sequences=True,
                                    name=layer_name, dropout=dropout)
        encoder_outputs, state_h, state_c = encoder(encoder_outputs, initial_state=[state_h, state_c])
  
  encoder_states = None

  # saving the last state
  if cell_type  == 'rnn' or cell_type == 'gru' :
      encoder_states = [state_h]
  else:
      encoder_states = [state_h, state_c]
  decoder_inputs = keras.Input(shape=(None,), name='decoder_input')
  de_embed = tf.keras.layers.Embedding(num_decoder_tokens, embedding_size, name='decoder_embedding')(decoder_inputs)
 # number of decoder layers
  d_layer = number_of_decoder_layers
  decoder = None

  # first decoder layer

  if cell_type == 'rnn':
      decoder = keras.layers.SimpleRNN(latent_dimension, return_sequences=True, return_state=True,
                                        name='decoder_hidden_1', dropout=dropout)
      
     # For all decoder layers, the initial state is the last state of the last encoder layer
      decoder_outputs, _ = decoder(de_embed, initial_state=encoder_states)
  elif cell_type == 'gru':
      decoder = keras.layers.GRU(latent_dimension, return_sequences=True, return_state=True,
                                  name='decoder_hidden_1', dropout=dropout)
      
      decoder_outputs, _ = decoder(de_embed, initial_state=encoder_states)
  else:
      decoder = keras.layers.LSTM(latent_dimension, return_sequences=True, return_state=True,
                                  name='decoder_hidden_1', dropout=dropout)
      
      decoder_outputs, _, _ = decoder(de_embed, initial_state=encoder_states)

  for i in range(2, d_layer + 1):
      layer_name = 'decoder_hidden_%d' % i
      if cell_type == 'rnn':
          decoder = keras.layers.SimpleRNN(latent_dimension, return_sequences=True, return_state=True,
                                            name=layer_name, dropout=dropout)
          decoder_outputs, _ = decoder(decoder_outputs, initial_state=encoder_states)
      elif cell_type == 'gru':
          decoder = keras.layers.GRU(latent_dimension, return_sequences=True, return_state=True,
                                      name=layer_name, dropout=dropout)
          decoder_outputs, _ = decoder(decoder_outputs, initial_state=encoder_states)
      else:
          decoder = keras.layers.LSTM(latent_dimension, return_sequences=True, return_state=True,
                                      name=layer_name, dropout=dropout)
          decoder_outputs, _, _ = decoder(decoder_outputs, initial_state=encoder_states)

  # add a dense layer
  decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax", name='decoder_output')
  decoder_outputs = decoder_dense(decoder_outputs)

  model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
  
  return model
  # # We discard `encoder_outputs` and only keep the states.
  # encoder_states = [state_h, state_c]

  # # Set up the decoder, using `encoder_states` as initial state.
  # decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

  # # We set up our decoder to return full output sequences,
  # # and to return internal states as well. We don't use the
  # # return states in the training model, but we will use them in inference.
  # decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
  # decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
  # decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
  # decoder_outputs = decoder_dense(decoder_outputs)

  # # Define the model that will turn
  # # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
  # model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
def fit(model,cell_type,encoder_input_data, decoder_input_data, decoder_target_data,batch_size, epochs,number_of_encoder_layers,number_of_decoder_layers,latent_dimension, callbacks=None):
  model.compile(optimizer="adam", loss="categorical_crossentropy",metrics=['accuracy'])
  model.fit(
      [encoder_input_data, decoder_input_data],
      decoder_target_data,
      batch_size=batch_size,
      epochs=epochs,
      callbacks=callbacks
  )

  # create inference model
  encoder_inputs = model.input[0]  # input_1
  if cell_type == 'rnn' or cell_type == 'gru':
      encoder_outputs, state_h_enc = model.get_layer('encoder_hidden_' + str(number_of_encoder_layers)).output
      encoder_states = [state_h_enc]
      encoder_model = keras.Model(encoder_inputs, encoder_states)

      decoder_inputs = model.input[1]  # input_2
      decoder_outputs = model.get_layer('decoder_embedding')(decoder_inputs)
      decoder_states_inputs = []
      decoder_states = []

      for j in range(1, number_of_decoder_layers + 1):
          decoder_state_input_h = keras.Input(shape=(latent_dimension,))
          current_states_inputs = [decoder_state_input_h]
          decoder = model.get_layer('decoder_hidden_' + str(j))
          decoder_outputs, state_h_dec = decoder(decoder_outputs, initial_state=current_states_inputs)
          decoder_states += [state_h_dec]
          decoder_states_inputs += current_states_inputs
  else:
      encoder_outputs, state_h_enc, state_c_enc = model.get_layer('encoder_hidden_'+ str(number_of_encoder_layers)).output
      encoder_states = [state_h_enc, state_c_enc]
      encoder_model = keras.Model(encoder_inputs, encoder_states)

      decoder_inputs = model.input[1]  # input_2
      decoder_outputs = model.get_layer('decoder_embedding')(decoder_inputs)
      decoder_states_inputs = []
      decoder_states = []

      for j in range(1,number_of_decoder_layers + 1):
          decoder_state_input_h = keras.Input(shape=(latent_dimension,))
          decoder_state_input_c = keras.Input(shape=(latent_dimension,))
          current_states_inputs = [decoder_state_input_h, decoder_state_input_c]
          decoder = model.get_layer('decoder_hidden_' + str(j))
          decoder_outputs, state_h_dec, state_c_dec = decoder(decoder_outputs, initial_state=current_states_inputs)
          decoder_states += [state_h_dec, state_c_dec]
          decoder_states_inputs += current_states_inputs
          
  decoder_dense = model.get_layer('decoder_output')
  decoder_outputs = decoder_dense(decoder_outputs)
  decoder_model = keras.Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

  return encoder_model , decoder_model

In [ ]:
# Reverse-lookup token index to decode sequences of numbers back to sequence of characters.

reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [ ]:
def decode_sequence(input_seq,number_of_decoder_layers,target_token_index,cell_type,
                    reverse_target_char_index,max_decoder_seq_length,
                    encoder_model,decoder_model):
  # Encode the input as state vectors.
  states_value = [encoder_model.predict(input_seq)]*number_of_decoder_layers

  # Generate empty target sequence of length 1.
  empty_seq = np.zeros((1, 1))

  # Populate the first character of target sequence with the start character.
  empty_seq[0, 0] = target_token_index["\t"]
  target_seq = empty_seq


  stop_condition = False
  decoded_sentence = ""
  while not stop_condition:
      if cell_type  == 'rnn' or cell_type == 'gru':
          temp = decoder_model.predict([target_seq] + [states_value])
          output_tokens, states_value = temp[0], temp[1:]
      else:
          temp = decoder_model.predict([target_seq] + states_value )
          output_tokens, states_value = temp[0], temp[1:]

      # Sample a token
      sampled_token_index = np.argmax(output_tokens[0, -1, :])
      sampled_char = reverse_target_char_index[sampled_token_index]
      decoded_sentence += sampled_char

      # Exit condition: Encounter stop character or be greater than the max decoder length
      
      if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
          stop_condition = True

      # Update the target sequence (of length 1).
      target_seq = np.zeros((1, 1))
      target_seq[0, 0] = sampled_token_index

  return decoded_sentence

In [ ]:
def accuracy(val_encoder_input_data, y_val,number_of_decoder_layers,target_token_index,cell_type,
                    reverse_target_char_index,max_decoder_seq_length,
                    encoder_model,decoder_model,verbose=False):
  n_correct = 0
  n_total = 0
  number_of_decoder_layers=number_of_decoder_layers
  for seq_index in range(len(val_encoder_input_data)):

    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = val_encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq,number_of_decoder_layers,target_token_index,cell_type,
                    reverse_target_char_index,max_decoder_seq_length,
                    encoder_model,decoder_model)
    # print("-")
    # print("Input sentence:", y_val[seq_index])
    # print("Decoded sentence:", decoded_sentence)

    if decoded_sentence.strip() == y_val[seq_index].strip():
        n_correct += 1

    n_total += 1

    if verbose:
        print('Prediction ', decoded_sentence.strip(), ',Ground Truth ', y_val[seq_index].strip())

  return n_correct * 100.0 / n_total

In [ ]:
#SWEEP
def sweep():
  run = wandb.init()
  config = wandb.config
  name = ('cell_type=%s' % config.cell_type) + "_" \
          + ('embedding=%d' % config.embedding_size) + "_" \
          + ('n_enc_layers=%d' % config.number_of_encoder_layers) + "_" \
          + ('n_dec_layers=%d' % config.number_of_decoder_layers) + "_" \
          + ('latent=%d' % config.latent_dimension) + "_" \
          + ('dropout=%.2f' % config.dropout) + "_" \
          + ('epochs=%d' % config.epochs)

  wandb.run.name = name
  batch_size = 64  # Batch size for training.
  epochs = config.epochs  # Number of epochs to train for.
  latent_dim = config.latent_dimension  # Latent dimensionality of the encoding space .#hidden states hyperparameter

  #Train the model
  model = create_model(num_encoder_tokens=num_encoder_tokens,embedding_size=config.embedding_size,
                      cell_type = config.cell_type,latent_dimension=config.latent_dimension,
                      dropout=config.dropout,number_of_encoder_layers = config.number_of_encoder_layers,
                      num_decoder_tokens=num_decoder_tokens,
                      number_of_decoder_layers=config.number_of_decoder_layers)
  
  encoder_model , decoder_model = fit(model=model,cell_type=config.cell_type,encoder_input_data=encoder_input_data, 
                                    decoder_input_data=decoder_input_data,
          decoder_target_data=decoder_target_data,batch_size=batch_size,epochs=config.epochs,
          number_of_encoder_layers = config.number_of_encoder_layers,
          number_of_decoder_layers= config.number_of_decoder_layers,latent_dimension=latent_dim, 
          callbacks=[WandbCallback()])
  #Validation accuracy

  subset = 20

  val_accuracy = accuracy(val_encoder_input_data, y_val,
                          number_of_decoder_layers=config.number_of_decoder_layers,
                          target_token_index=target_token_index,cell_type=config.cell_type,
                          reverse_target_char_index=reverse_target_char_index,max_decoder_seq_length=max_decoder_seq_length,
                          encoder_model=encoder_model,decoder_model=decoder_model
                          )
  print('Validation accuracy: ', val_accuracy)
  wandb.log({'val_accuracy': val_accuracy})
  run.finish()
  

In [ ]:
# run sweeps
sweep_config = {
    'method': 'bayes',  # grid, random
    'metric': {
        'name': 'val_accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'epochs': {
            'values': [20,25,30]
        },
        'embedding_size': {
            'values': [16,32,64,128,256]
        },
        'number_of_encoder_layers': {
            'values': [1,2,3]
        },
        'number_of_decoder_layers': {
            'values': [1,2,3]
        },
        'latent_dimension': {
            'values': [16, 32, 64, 256, 512]
        },
        'cell_type': {
            'values': ['gru','lstm','rnn']
        },                             
        'dropout': {
            'values': [0.0,0.2,0.3,0.4,0.5]
        }
    }
}

In [ ]:
#start wandb sweep
sweep_id = "emcrqbu0"
wandb.agent(sweep_id, sweep, project="CS6910-Assignment_3-sweep-Tamil",entity="hithesh-sidhesh",count = 5)


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: 58r6tft4 with config:
wandb: 	cell_type: rnn
wandb: 	dropout: 0
wandb: 	embedding_size: 64
wandb: 	epochs: 20
wandb: 	latent_dimension: 16
wandb: 	number_of_decoder_layers: 2
wandb: 	number_of_encoder_layers: 1


Epoch 1/20
1066/1066 [==============================] - 60s 52ms/step - loss: 1.0760 - accuracy: 0.7403 - _timestamp: 1651301659.0000 - _runtime: 67.0000
Epoch 2/20
1066/1066 [==============================] - 55s 52ms/step - loss: 0.7425 - accuracy: 0.7895 - _timestamp: 1651301715.0000 - _runtime: 123.0000
Epoch 3/20
1066/1066 [==============================] - 55s 52ms/step - loss: 0.6812 - accuracy: 0.8061 - _timestamp: 1651301770.0000 - _runtime: 178.0000
Epoch 4/20
1066/1066 [==============================] - 55s 51ms/step - loss: 0.6476 - accuracy: 0.8157 - _timestamp: 1651301824.0000 - _runtime: 232.0000
Epoch 5/20
1066/1066 [==============================] - 55s 51ms/step - loss: 0.6293 - accuracy: 0.8198 - _timestamp: 1651301879.0000 - _runtime: 287.0000
Epoch 6/20
1066/1066 [==============================] - 55s 51ms/step - loss: 0.6169 - accuracy: 0.8232 - _timestamp: 1651301934.0000 - _runtime: 342.0000
Epoch 7/20
1066/1066 [==============================] - 55s 51ms/step -

wandb: Ctrl + C detected. Stopping sweep.
wandb: Ctrl + C detected. Stopping sweep.


In [ ]:
# sweep_id = wandb.sweep(sweep_config, project="CS6910-Assignment_3-sweep-Tamil")
# #start wandb sweep
# wandb.agent(sweep_id, sweep, project="CS6910-Assignment_3-sweep-Tamil",count = 2)